In [ ]:
#import libraries
import os
import cv2
import json
import re

In [ ]:
# local variables

p3225_file_path = {
    "video folder" : "DI_CAMERA_P3225",
    "1st level" : ["Final"],
    "2nd level" : "Patient Name",
    "3rd level" : [2, 3],
    "4th level" : { "B": "With Blankets",
                        "WOB": "Without Blankets"}, 
    "5th level" : {"H" : "Hold Breath",
                    "R": "Relaxed"} 
}

fur_file_path = {
    "video folder" : "DI_THERMAL_FLIR",
    "1st level" : [""],
    "2nd level" : "Patient Number",
    "3rd level" : [2, 3],
    "4th level" : { "H": "Hold Breath",
                        "R": "Relaxed"}, 
    "5th level" : { "B": "With Blankets",
                        "WOB": "Without Blankets"} 
}

root_path = "E:/Christian/DI_centre_structured"
repo_dir = os.getcwd()

1st stage:
1) get mp4/avi filenames and store results in xlsx file
2) rename mp4/avi filenames to a common convention

Workflows:
- Get all mp4/avi filenames in the folder (ignore filenames that ends with amp-.mp4)
- Record and keep track of all filenames 
- Manually check ambiguous cases

2nd stage:
1) extract metadata
2) edit videos so that frame rates are a decimal value

Workflows:
- extract and record metadata
- interpolate and extrapolate data
- decide on a common frame

3nd stage:
1) turn all mp4/avi files to jpg using opencv
2) pick out an "even x" amount of jpg files in the folder

Workflows:
- convert all mp4 files into frames
- Pick out desired frames
- avoid any memory allocation issues

4th stage:
1) automate manual tasks

Workflows:
- track changes in metadata

Notes:
- metadata manually recorded for now and will automate later

Questions:
- metadata don't match

In [ ]:
def get_video_filenames(patient_path:str, level=0) -> None:
    file_paths = []

    os.chdir(patient_path)
    if level < 4:
        for entry in os.scandir(patient_path):
            if entry.is_file():
                file_paths.append(entry.path)
            elif entry.is_dir():
                file_paths.extend(get_video_filenames(entry.path, level+1))

    return file_paths

In [ ]:
def get_patient_id(curr_dir:str) -> None:
    os.chdir(curr_dir)
    patient_names = [folder_content for folder_content in os.listdir() if not folder_content.endswith(".mp4")]
    return patient_names

In [ ]:
def filter_filenames(video_filenames:list[str], keyword:str) -> list[str]:
    filtered_filenames = []
    for filename in video_filenames:
        if not re.search(r"\b{}\b".format(keyword), filename, flags=re.IGNORECASE):
            filtered_filenames.append(filename)
    return filtered_filenames

In [ ]:
def save_filenames(root_dir:str, save_dir:str, video_name:str, patient_list:list[str]) -> None:
    log_file = f"{save_dir}/{video_name}.log"
    log_file = log_file.replace("\\", "/")

    if os.path.exists(log_file):
        os.remove(log_file)

    for patient_count, patient_name in enumerate(patient_list):
        patient_folder = root_dir + "/" + patient_name
        all_patient_mp4_files = get_video_filenames(patient_folder)
        filtered_filenames = filter_filenames(all_patient_mp4_files, "media_media.amp-.mp4")

        with open(log_file, "a") as mp4_data:
            for mp4_file in filtered_filenames:
                mp4_data.write(f"{mp4_file}\n")

        if patient_count == 1:
            break

In [ ]:
""" 
    get filenames for rgb camera
"""

video_folder = p3225_file_path["video folder"]
all_videos_folder = root_path + "/" + video_folder + "/" + p3225_file_path["1st level"][0]
patient_names = get_patient_id(all_videos_folder)
patient_names

In [ ]:
records_folder = repo_dir + "/record"
save_filenames(all_videos_folder, records_folder, video_folder, patient_names)

# FLIR mp4 files

In [ ]:
root_path = "C:/Users/chris/Documents/Projects Outside of School/Research w Dr S and Illia/DI_centre_structured/DI_THERMAL_FLIR"

patient_numbers = [str(i) for i in range(1, 19)]
patient_numbers

In [ ]:
"""
    TODO: combine this function with save_mp4
"""

def save_avi_filenames_to_txt_file(root_dir, save_dir, patient_list):
    txt_file = save_dir + "/avi_filenames.txt"
    txt_file = txt_file.replace("\\", "/")

    for patient_count, patient_name in enumerate(patient_list):
        patient_folder = root_dir + "/" + patient_name
        all_patient_mp4_files = get_patient_mp4_files(patient_folder)
        wanted_patient_mp4_files = [mp4_file for mp4_file in all_patient_mp4_files if not mp4_file.replace(" ", "").endswith(".Bmp") and not mp4_file.replace(" ", "").endswith(".png") ]

        with open(txt_file, "a") as mp4_data:
            for mp4_file in wanted_patient_mp4_files:
                mp4_data.write(f"{mp4_file}\n")

In [ ]:
save_avi_filenames_to_txt_file(root_path, records_folder, patient_numbers)

In [ ]:
"""
    A function to convert mp4 files to frames
"""

def convert_mp4_to_frames(mp4_file:str, output_dir:str):
    video = cv2.VideoCapture(mp4_file)
    os.makedirs(output_dir, exist_ok=True)
    frame_count = 0
    not_end_of_vid = True

    while not_end_of_vid:
        ret, frame = video.read()
        
        if not ret:
            not_end_of_vid = False
            break

        frame_file = os.path.join(output_dir, f"frame{frame_count:05d}.jpg")
        cv2.imwrite(frame_file, frame)
        frame_count += 1

    video.release()

    print(f"Conversion complete: {frame_count} frames saved in {output_dir}")

In [ ]:
"""
    display video properties using opencv
"""

def get_video_properties(mp4_file: str) -> dict:
    video_properties = {}
    video = cv2.VideoCapture(mp4_file)
    old_fps = int(video.get(cv2.CAP_PROP_FPS))
    number_of_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    video_properties["old fps"] = old_fps
    video_properties["frames"] = number_of_frames


    return video_properties

In [ ]:
""" testing the function convert_mp4_to_frames(mp4_file:str, output_dir:str)"""
mp4_file = repo_dir + "/sample/Arun2.mp4"
output_directory = repo_dir + "/sample/frames"

# convert_mp4_to_frames(mp4_file, output_directory)

In [ ]:
""" testing the function convert_mp4_to_frames(mp4_file:str, output_dir:str)"""
avi_file = repo_dir + "/sample/15.avi"
output_directory = repo_dir + "/sample/15_frames"

convert_mp4_to_frames(avi_file, output_directory)

In [ ]:
""" testing the function get_video_properties(mp4_file: str)"""
video_properties = get_video_properties(mp4_file)
video_properties_json = json.dumps(video_properties, indent=2)
print(video_properties_json)